In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import pandas as pd
import glob
import os
from IPython.display import display
import matplotlib.pyplot as plt
import json
from sklearn import preprocessing
from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding
from keras.preprocessing.text import Tokenizer
import tensorflow_addons as tfa
import decimal
from sklearn.preprocessing import normalize
from sklearn.preprocessing import MinMaxScaler
import math
from tensorflow.keras import activations
import itertools

In [2]:
trailers_csv = "/Volumes/Seagate/natasha-diploma/trailers.csv"
trailers_df = pd.read_csv(trailers_csv, index_col=None, header=0)
trailers_df.drop(trailers_df.columns[trailers_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(trailers_df.head())

,duration,release_date,restrict,orig_title,kinopoisk_id,imdb_rating,kinopoisk_rating,synopsis,categories,genres,...,emotions,Place of act,Qualities,Time of act,Based on,Audience,Mood,Subgenre,About,Theme
0,5816.0,2019-04-05,18.0,Phil,960568,5.4,5.86,"Фил – стоматолог, страдающий от депрессии. Его...",{Фильмы},"{Драмы,Зарубежные,Комедии}",...,"{'neutral': 0.5312193632125854, 'negative': 0....",['США'],['дебютный'],"['21-й век', 'новейшее время']",['оригинального сценария'],"['для молодёжи', 'для взрослых']","['грустный', 'депрессивный', 'трогательный']","['Драмы', 'Комедийные']","['преследования', 'самозванцы', 'расследование...","['развлекательные', 'психологический']"
1,5123.0,2019-04-02,18.0,Tooth Fairy,1126022,2.2,0.00,Старая женщина начинает рассказывать легенду о...,{Фильмы},"{Зарубежные,Триллеры,Ужасы}",...,"{'negative': 0.4688006341457367, 'skip': 0.392...",['не определено'],"['зрелищные', 'жанровый', 'неожиданный финал']",['не определено'],"['эпоса и легенд', 'оригинального сценария']","['для взрослых', 'для мужчин', 'для подготовле...","['трагичные', 'мрачные', 'шокирующие', 'захват...","['ужасы', 'триллер']","['преследования', 'страхи', 'дети', 'молодежь'...",['развлекательные']
2,4667.0,2019-01-19,16.0,Wake Up,1069713,2.9,0.00,"Врач-психиатр исследует дневник девушки, котор...",{Фильмы},"{Боевики,Зарубежные,Криминал,Триллеры,Ужасы}",...,"{'positive': 0.3140605390071869, 'negative': 0...",['не определено'],"['жанровый', 'дебютный']",['новейшее время'],['оригинального сценария'],"['для взрослых', 'для подготовленного зрителя'...","['захватывающие', 'жестокие', 'напряженные', '...","['Драмы', 'Боевики', 'триллер']","['психотерапевты', 'кровь', 'тайна смерти', 'п...","['психологический', 'развлекательные']"
3,6148.0,2019-08-09,16.0,The Kitchen,1117988,5.5,5.50,"Смотрите в 4К! Три жены ирландских гангстеров,...",{Фильмы},"{Боевики,Драмы,Зарубежные,Криминал,По комиксам}",...,"{'neutral': 0.4765896201133728, 'negative': 0....","['большой город', 'США']","['жанровый', 'остросюжетные']",['20-й век'],['комиксов'],"['для мужчин', 'для женщин', 'для взрослых']","['захватывающие', 'авантюрный', 'жестокие', 'н...","['Боевики', 'Драмы', 'криминальные']","['враги или вражда', 'женщины', 'антигерои', '...","['авантюрные', 'гангстерская']"
4,6013.0,2019-05-02,12.0,Pokémon Detective Pikachu,994864,6.6,6.56,Смотрите в 4K! Элитный детектив Пикачу отправл...,"{Для детей,Фильмы}","{Детективы,Зарубежные,Комедии,Семейные,Фильмы,...",...,"{'neutral': 0.4532718360424042, 'negative': 0....","['вымышленная страна (город)', 'большой город']","['частично анимационный', 'зрелищные', 'кассов...",['21-й век'],"['оригинального сценария', 'сериалов']","['для детей 13-16 лет', '9 лет', '8 лет', '7-1...","['авантюрный', 'захватывающие', 'динамичный', ...","['детские', 'Приключенческие', 'Фантастические...","['притяжение противоположностей', 'риск', 'пре...","['развлекательные', 'образовательный']"


In [3]:
sessions_csv = "/Volumes/Seagate/natasha-diploma/sessions.csv"
sessions_df = pd.read_csv(sessions_csv, index_col=None, header=0)
sessions_df.drop(sessions_df.columns[sessions_df.columns.str.contains('Unnamed',case = False)],axis = 1, inplace = True)
display(sessions_df)

,0,1,2,3,4,5,6,7,8,9
0,403a42b604f504069c41d39c5a4fc2,23ee5640ec588ffaa78afa3268b04a,5b150a1cae009459493bd0fe5c7b3e,e291f8ae1de4a38c71027e9c91d51c,34cfd44ad5cc2d63441ebf437eccb5,02fbcf93a730c9c9d5e65aef41c67e,26b3f811a6b9ed75d3cc5198105fa4,6fcff414343456465369b05f6851eb,e2f15819192cded77ed1825a129c0c,41b772a5b18bc724562bfb60f4f3ac
1,a5b2dd3ed41aabb5733c7cfffff0bc,25e573b81ab69d6cf700a51281fc2a,49c58f679f72e0f02b97bdeee1a1bb,c0be492c14bcec5ab3dfd34d95d6f5,b2274b9b4c8c4dc2f79d417ddd71e5,7fd08607f52b89a753437eee177e32,25d41576f3529994c878a5d15f6dc7,ece16e15b2286f8287e8935f12b6c2,806c0ed32e135afb8db4f3aad71acf,f10b4adf1c732b6b260791fb24f430
2,80870ec50515107ba7034ab758cebe,cd522afebcf91625c3cd811f3fb6e2,061a59839837a123a916a70947f075,07e873c5aac8ffaf97897fbd0dfbf6,60fb638ff7ab469be8637111fdc837,d49c4a9b748b9c7abb9d98cdecebef,b0b062dc25c4dada316183941b100c,c7f809aab571e9acdf799a78c5ef61,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf
3,d66440e7524c0a8b9b206fbb7295ba,046f93c91d0befd4625f00e07eb473,199cf6d89a2f06fc552ebf86105d7d,2adfc14bbc0fba5b5d616d6fcfe3de,a458a5a362af9a7774780eb4e25365,88bbc5912f946b1fb77fdeaa3f5d4a,6d0bb6959972ac92c10d7445982a94,274869b912b256df69b9e7c5c0cb11,2c5bf20534b80ee47bed9a90b405a0,f456635bb3d2567a3a61133fa9dd89
4,231b80c884b2f1390e17f09a187889,bd6707bb04cfb8880a5f783c85be20,bbd5df907c099d2cc2b5285b08038e,514f3ee1f00df05dc3900251972245,961c336b0c2fa04c883c639dfee1ba,d6159553e41857fe23df9d3e746284,d2cd45a49d53702df403cffddd53e6,c0314a446fbef65ce9626b037a70da,d6e8f6d21631149a90ac940d3ee227,e762f208419b7449c0731037262bb3
...,...,...,...,...,...,...,...,...,...,...
196,91e8db1b91acc7bf903bf670461db3,f65c8906c07b627a760a0671de7144,26ba677f7ff11ac529f5f69c54cb0b,be6360babdb2e5910c5bc6f4f04f0d,705c23ed5ae1bc8824fc955b82094b,74733260a8a5143b2eb20c4a62940a,afed0e34e5af6958fef610a0ac5565,0a49f9b4576a63be41cb96908eba5e,baec94926106f54090568b653e84b4,8a2b5f96cc30660b28636fbe2664d5
197,c718bcdf6240802ba7123a7a631199,5e62a64d4f0d6ca3fce397267bf9f3,c5d40f1adeb446df77e65815da405d,bbf4b4a3db602ab9b88845114f7244,0f462e6bbd719e012902d8814d0e35,e5bca888fec7f8c0116db0002a7077,8a2483e11828332a9e2fc8659ae38f,bb59c82aeb5e44eb5735d5a1c016b8,8d3877a1b26b10642d125ec4fc4c9b,ae90166aa81ab2bf878dace61a5620
198,ee78a3aae1a69f2e4b82f26d41d594,11fe2dd4cccaf0567c878e19e1811a,aab18e84a97780bafb188fd33cb2a9,838cb9d91dd080147bcc18982bcdbb,972821d03656142f33707487f51b66,d6d45793f5b093e8bdf3d8ae4952b6,28731720ee3330b7107cec47168b8c,dad45a9302ad916f6901b86a744cdd,644bf6d0bdf6c9cddf71f5ab4cf7d4,4774caaf01df3d5d982e0cb77b1a13
199,b0b062dc25c4dada316183941b100c,80870ec50515107ba7034ab758cebe,d49c4a9b748b9c7abb9d98cdecebef,67c38de267396951227897efc503e1,7b457a5d30f3cb01b6998390ec7abf,67f32aff5488778cb4bbb6b21acc7d,3d558c3e37ccb70937b171aeb67bbb,97474799458a67e30a70f66407420b,0dbe493275a25285829b979316d566,ba54f9d576e93f2b0cf4622a92310f


In [4]:
movies_id = sessions_df.to_numpy()
movies_id = np.array(movies_id).flatten()
vocab = np.unique(movies_id)
vocab_len = len(vocab)

# print(vocab)
print(vocab_len)
print(movies_id.shape)

366
(2010,)


In [5]:
def getVideoInfo(info_df):
    return [
        np.mean(info_df['temperature'].to_numpy()),
        np.mean(info_df['brightness'].to_numpy()),
        np.mean(info_df['colorfulness'].to_numpy()),
        np.mean(info_df['energy'].to_numpy()),
        np.mean(info_df['tempo'].to_numpy()),
        np.mean(info_df['amplitude'].to_numpy()),
        np.mean(info_df['spectral_rolloff'].to_numpy()),
        np.mean(info_df['mfcc'].to_numpy())
    ]

In [6]:
info = {}
scenes_array = []
del_array_name = []
info_folder = "/Volumes/Seagate/natasha-diploma/videoinfo"
for i in range(0, len(movies_id)):
    try:
        info_csv = info_folder + '/' + movies_id[i] + '.csv'
        info_df =  pd.read_csv(info_csv, index_col=None, header=0)
        info[movies_id[i]] = getVideoInfo(info_df.iloc[: , 4:])
        scenes_array.append(info[movies_id[i]])
    except:
        del_array_name.append(movies_id[i])

In [7]:
def vocab_creater(arr):
    word2idx = {}
    idx2word = {}
    for i in range(len(arr)):
        word2idx[arr[i]] = i
        idx2word[i] = arr[i]

    return word2idx, idx2word, len(arr)

In [8]:
word2idx, idx2word, _ = vocab_creater(vocab)

In [9]:
def toArray(smth):
    new = []
    
    for i in range(len(smth)):
        new.append(word2idx[smth[i]])
    
    return new


tokenized = toArray(movies_id)

In [10]:
print(np.array(scenes_array).shape)
print(np.array(tokenized).shape)

scenes_array = np.nan_to_num(scenes_array, copy=False, nan=0.0, posinf=0.0, neginf=0.0)
scenes_array = normalize(scenes_array, axis=1, norm='l1')

scaler = MinMaxScaler((-1, 1))
scenes_array = np.around(scaler.fit_transform(scenes_array), 3)

print(scenes_array)

(2010, 8)
(2010,)
[[ 1.    -1.    -1.    ...  1.    -1.     0.633]
 [-1.     0.455 -0.021 ...  0.25   0.957  0.364]
 [ 1.    -1.    -1.    ...  1.    -1.     0.633]
 ...
 [-1.     0.375 -0.405 ...  0.184  0.98  -0.463]
 [ 1.    -1.    -1.    ...  1.    -1.     0.633]
 [ 1.    -1.    -1.    ...  1.    -1.     0.633]]


In [11]:
id_size = 1
id_embed_size = 8
vocab_size = 366
embedding_dim = 1024
rnn_units = 1024

In [12]:
class MyModel(tf.keras.Model):
    def __init__(self, vocab_size, id_size, id_embed_size, embedding_dim, rnn_units):
        super().__init__(self)
        self.id_embedding = tf.keras.layers.Embedding(vocab_size, id_embed_size)
        self.normalization = tf.keras.layers.BatchNormalization()
        self.concat = tf.keras.layers.Concatenate()
        self.gru = tf.keras.layers.GRU(rnn_units,
                                       return_sequences=True,
                                       return_state=True)
        self.dense = tf.keras.layers.Dense(vocab_size, activation=activations.relu)

    def call(self, _input, states=None, return_state=False, training=False):
        ids = self.id_embedding(_input[0])
        features = self.normalization(_input[1])
        x = self.concat([ids, features])
        if states is None:
            states = self.gru.get_initial_state(x)
        x, states = self.gru(x, initial_state=states, training=training)
        x = self.dense(x, training=training)

        if return_state:
            return x, states
        else:
            return x
        
#     def train_step(self, data):
#         with tf.GradientTape() as tape:
#             y_pred = self(data, training=True)
#             loss = self.compiled_loss(y, y_pred, regularization_losses=self.losses)

#         trainable_vars = self.trainable_variables
#         gradients = tape.gradient(loss, trainable_vars)
#         self.optimizer.apply_gradients(zip(gradients, trainable_vars))
#         self.compiled_metrics.update_state(y, y_pred)

#         return {m.name: m.result() for m in self.metrics}

In [13]:
train_tokenized = tokenized[:1400]
test_tokenized = tokenized[1400:2000]

train_scenes = scenes_array[:1400]
test_scenes = scenes_array[1400:2000]

ids_ds = tf.data.Dataset.from_tensor_slices(train_tokenized)
features_ds = tf.data.Dataset.from_tensor_slices(train_scenes)

ids_ds_test = tf.data.Dataset.from_tensor_slices(test_tokenized)
features_ds_test = tf.data.Dataset.from_tensor_slices(test_scenes)

ds_test = tf.data.Dataset.zip((ids_ds_test, features_ds_test))

ds = tf.data.Dataset.zip((ids_ds, features_ds))
seq_length = 10
examples_per_epoch = len(tokenized) // (seq_length + 1)

sequences = ds.batch(seq_length + 1, drop_remainder=True)
ids_seq = ids_ds.batch(seq_length + 1, drop_remainder=True)
features_seq = features_ds.batch(seq_length + 1, drop_remainder=True)
sequences_test = ds_test.batch(seq_length + 1, drop_remainder=True)

ds

<ZipDataset shapes: ((), (8,)), types: (tf.int32, tf.float64)>

In [14]:
def full_split_input_target(id_sequence, feature_sequence):
    input_text = (id_sequence[:-1], feature_sequence[:-1])
    target_text = (id_sequence[1:])
    return input_text, target_text

def split_input_target(seq):
    input_text = seq[:-1]
    target_text = seq[1:]
    return input_text, target_text

ds = sequences.map(full_split_input_target)
ids_ds = ids_seq.map(split_input_target)
features_ds = features_seq.map(split_input_target)
ds_test = sequences_test.map(full_split_input_target)

BATCH_SIZE = 32
BUFFER_SIZE = 10000

ds = (
    ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))


ids_ds = (
    ids_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

features_ds = (
    features_ds
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

for _input, _output in ds_test.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True).take(1):
    print(_input)

ds_test = (
    ds_test
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE))

ds

(<tf.Tensor: shape=(32, 10), dtype=int32, numpy=
array([[357, 141, 332,  90,   8,  64, 166,  33, 179,  19],
       [365, 313,  41, 331,  26,  10, 109, 166,  61,  86],
       [ 53, 104,  60, 141, 332, 272, 274, 228,  99, 162],
       [ 60, 130, 284, 240,  28,  89,  40, 134, 173, 128],
       [355,  65, 286, 169, 176, 257,  23, 276, 206, 296],
       [154, 290,   7, 315, 145,  88, 269,  43, 127, 205],
       [231,  53, 104,  60, 141, 332,  90, 354, 201,  76],
       [267, 165, 224, 223, 327, 119, 156, 236, 235, 101],
       [108, 347, 262,  60, 104, 298, 159, 231, 364, 222],
       [141, 332,  90, 299, 192, 259, 161, 312,  85, 196],
       [285, 279, 132, 225, 317, 310, 289, 320, 311,  96],
       [159,  60, 141, 117,  67,  22,  44,  77, 325, 253],
       [258, 271, 213, 143, 182, 156, 223, 224, 165, 119],
       [194, 227, 319,  70, 324, 157, 118, 259, 192, 312],
       [294, 280,  29, 338, 205, 277, 212, 254, 349,  35],
       [ 38, 344, 125, 222,  58, 318, 188, 364, 126,  32],
       

<PrefetchDataset shapes: (((32, 10), (32, 10, 8)), (32, 10)), types: ((tf.int32, tf.float64), tf.int32)>

In [15]:
for _input, _output in ds_test.take(1):
    print(_input, _output)

(<tf.Tensor: shape=(32, 10), dtype=int32, numpy=
array([[294, 280,  29, 338, 205, 277, 212, 254, 349,  35],
       [231,  53, 104,  60, 141, 332,  90, 354, 201,  76],
       [251,  15, 142, 285, 283, 279,  59, 225, 132, 317],
       [147, 164,  45,  58,  57,  97, 199,   9, 201, 242],
       [206, 330, 229, 314,  94, 274, 222,  27, 104, 159],
       [335, 332, 141,  90,   8,  64, 166, 333, 108, 347],
       [ 85, 312, 196, 259, 184, 161, 181, 162, 299, 192],
       [ 60, 130, 284, 240,  28,  89,  40, 134, 173, 128],
       [ 64, 166, 333,  10, 365, 331, 313,  41, 186, 275],
       [141, 332,  90, 299, 192, 259, 161, 312,  85, 196],
       [357, 141, 332,  90,   8,  64, 166,  33, 179,  19],
       [175, 342, 110, 254, 338, 205,  63, 243, 121, 291],
       [ 24,  92, 180,   1, 195, 256, 135,  47, 318,  26],
       [110, 338, 205, 277, 114, 120, 307, 123, 142, 101],
       [365, 313,  41, 331,  26,  10, 109, 166,  61,  86],
       [225, 317, 310, 289, 320,  48, 265, 270, 175, 342],
       

In [16]:
features_ds

<PrefetchDataset shapes: ((32, 10, 8), (32, 10, 8)), types: (tf.float64, tf.float64)>

In [17]:
model = MyModel(
    vocab_size=vocab_size,
    id_size=id_size,
    id_embed_size=id_embed_size,
    embedding_dim=embedding_dim,
    rnn_units=rnn_units)


In [18]:
for _input, _output in ds.take(1):
    example_batch_predictions = model(_input)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")
    sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
    sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

    print(sampled_indices)

(32, 10, 366) # (batch_size, sequence_length, vocab_size)
[  6 278  73 290 330 341 274 281  50 284]


In [19]:
tf.config.run_functions_eagerly(True)
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  2928      
_________________________________________________________________
batch_normalization (BatchNo multiple                  32        
_________________________________________________________________
concatenate (Concatenate)    multiple                  0         
_________________________________________________________________
gru (GRU)                    multiple                  3201024   
_________________________________________________________________
dense (Dense)                multiple                  375150    
Total params: 3,579,134
Trainable params: 3,579,118
Non-trainable params: 16
_________________________________________________________________


In [20]:
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [21]:
model.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

In [22]:
EPOCHS = 200
history = model.fit(ds_test, epochs=EPOCHS)

Epoch 1/200
1/1 [==============================] - 0s 249ms/step - loss: 5.9030 - accuracy: 0.0000e+00
Epoch 2/200
1/1 [==============================] - 0s 237ms/step - loss: 5.8727 - accuracy: 0.0344
Epoch 3/200
1/1 [==============================] - 0s 238ms/step - loss: 5.8597 - accuracy: 0.0250
Epoch 4/200
1/1 [==============================] - 0s 246ms/step - loss: 5.8182 - accuracy: 0.0562
Epoch 5/200
1/1 [==============================] - 0s 240ms/step - loss: 5.7771 - accuracy: 0.0688
Epoch 6/200
1/1 [==============================] - 0s 240ms/step - loss: 5.7392 - accuracy: 0.0625
Epoch 7/200
1/1 [==============================] - 0s 230ms/step - loss: 5.7107 - accuracy: 0.0594
Epoch 8/200
1/1 [==============================] - 0s 232ms/step - loss: 5.6168 - accuracy: 0.0625
Epoch 9/200
1/1 [==============================] - 0s 249ms/step - loss: 5.5483 - accuracy: 0.0437
Epoch 10/200
1/1 [==============================] - 0s 229ms/step - loss: 5.4308 - accuracy: 0.0406
Epoch

Epoch 83/200
1/1 [==============================] - 0s 204ms/step - loss: 1.5844 - accuracy: 0.7437
Epoch 84/200
1/1 [==============================] - 0s 211ms/step - loss: 1.5953 - accuracy: 0.7281
Epoch 85/200
1/1 [==============================] - 0s 243ms/step - loss: 1.3568 - accuracy: 0.7875
Epoch 86/200
1/1 [==============================] - 0s 238ms/step - loss: 1.4350 - accuracy: 0.7531
Epoch 87/200
1/1 [==============================] - 0s 220ms/step - loss: 1.4167 - accuracy: 0.7656
Epoch 88/200
1/1 [==============================] - 0s 207ms/step - loss: 1.5623 - accuracy: 0.7406
Epoch 89/200
1/1 [==============================] - 0s 229ms/step - loss: 1.7340 - accuracy: 0.7344
Epoch 90/200
1/1 [==============================] - 0s 228ms/step - loss: 1.6199 - accuracy: 0.7406
Epoch 91/200
1/1 [==============================] - 0s 219ms/step - loss: 1.5299 - accuracy: 0.7344
Epoch 92/200
1/1 [==============================] - 0s 213ms/step - loss: 1.6204 - accuracy: 0.7281


1/1 [==============================] - 0s 208ms/step - loss: 0.8798 - accuracy: 0.8375
Epoch 165/200
1/1 [==============================] - 0s 207ms/step - loss: 0.9877 - accuracy: 0.8313
Epoch 166/200
1/1 [==============================] - 0s 201ms/step - loss: 1.1258 - accuracy: 0.7969
Epoch 167/200
1/1 [==============================] - 0s 204ms/step - loss: 0.9497 - accuracy: 0.8250
Epoch 168/200
1/1 [==============================] - 0s 203ms/step - loss: 1.0154 - accuracy: 0.8250
Epoch 169/200
1/1 [==============================] - 0s 202ms/step - loss: 1.0659 - accuracy: 0.8031
Epoch 170/200
1/1 [==============================] - 0s 202ms/step - loss: 0.9637 - accuracy: 0.8344
Epoch 171/200
1/1 [==============================] - 0s 208ms/step - loss: 1.0923 - accuracy: 0.8094
Epoch 172/200
1/1 [==============================] - 0s 202ms/step - loss: 0.9422 - accuracy: 0.8406
Epoch 173/200
1/1 [==============================] - 0s 208ms/step - loss: 1.0339 - accuracy: 0.8188
Epoc

In [23]:
ds_test

<PrefetchDataset shapes: (((32, 10), (32, 10, 8)), (32, 10)), types: ((tf.int32, tf.float64), tf.int32)>

In [24]:
model.evaluate(ds_test, batch_size=64, return_dict=True)

1/1 [==============================] - 0s 89ms/step - loss: 0.9076 - accuracy: 0.8500


{'loss': 0.9075837135314941, 'accuracy': 0.8500000238418579}